In [2]:
import yaml
import os
import shutil
import hls4ml
import matplotlib.pyplot as plt
%matplotlib inline
import sklearn.metrics as metrics
import numpy as np
import xmltodict

In [3]:
# Folder Creation
test_data_dir = 'TestData'
os.makedirs(test_data_dir, exist_ok=True)

syn_dir = 'Synthesis'
os.makedirs(syn_dir, exist_ok=True)

model_files_dir = 'ModelFiles'
os.makedirs(model_files_dir, exist_ok=True)

In [ ]:
# ROC Curve Function
def roc(model, test_in, test_truth):
    try:
        model_iter = iter(model)
    except TypeError:
        model_iter = iter([model])
    
    fig, ax = plt.subplots(figsize=(8,8))
    plt.grid(which='both')
    plt.ylim([0.0, 1.0])
    plt.xlim([0.0, 1.0])
    plt.ylabel("True Positive Rate")
    plt.xlabel("False Positive Rate")
    
    for m in model_iter:
        p = m.predict(test_in)[:,0]
        fpr, tpr, threshold = metrics.roc_curve(test_truth, p, drop_intermediate=False)
        auc = metrics.auc(fpr, tpr)
        x = np.linspace(0,1,1000)
        y = np.interp(x=x, xp=fpr, fp=tpr)
        plt.plot(x, y, linewidth=3, label=f'{m.name} (AUC={auc:.4f})')
        
    plt.plot([0, 1], [0, 1],color='r', linestyle='--', linewidth=3, label='Coin Flip')
    plt.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
# Model training wrapper
# Trains multiple models and returns the one with the best auc metric
def train_best_auc(model, train_in, train_truth, test_in, test_truth, training_passes=10, epochs=10, batch_size=32, callbacks=[], skip_reload=False):
    path = f'{model_files_dir}/{model.name}.h5'
    if os.path.exists(path) and not skip_reload:
        return keras.models.load_model(path)
    else:
        best_auc = 0
        best_model = None
        for i in range(training_passes):
            m = keras.models.clone_model(model)
            m.compile(optimizer='Nadam', loss=keras.losses.binary_crossentropy, metrics=[keras.metrics.BinaryAccuracy(), keras.metrics.AUC()])
            m.fit(train_in, train_truth, epochs=epochs, batch_size=batch_size, validation_data=(test_in, test_truth), callbacks=callbacks)
            p = m.predict(test_in)[:,0]
            fpr, tpr, threshold = metrics.roc_curve(test_truth, p)
            auc = metrics.auc(fpr, tpr)
            print(f'auc = {auc}' + '\n' + ('-'*50))
            if auc > best_auc:
                best_auc = auc
                best_model = m
        
        best_model.save(path)
        return best_model

In [ ]:
def calculate_metrics(y_true, inference_results_path):
    with open(inference_results_path, 'r') as fin:
        y_pred = np.array(fin.read().split('\n')[:len(y_true)]).astype(float)
    
    binary_accuracy = len(np.where(((y_true==1)&(y_pred>=0.5))|((y_true==0)&(y_pred<0.5)))[0])/len(y_true)
    
    fpr, tpr, threshold = metrics.roc_curve(y_true, y_pred, drop_intermediate=False)
    auc = metrics.auc(fpr, tpr)
    
    return binary_accuracy, auc

In [11]:
def parse_sim_rpt(report_path):
    with open(report_path, 'r') as fin:
        sim_rpt = {}
        for line in fin:
            key, value = line.split('=')
            key = key.replace('$', '').replace(' ', '')
            value = int(value.replace('\n', '').replace(' ', '').replace('"', ''))
            sim_rpt[key] = value
    
    return sim_rpt

In [1]:
# Synthesize HLS Model
def synthesize_model(model, io_type, ap_type, strategy, csim=False, cosim=False,
                     validate_sim_results=False, show_quantization_plots=False,
                     test_input=None, test_truth=None, delete_files=False):
    # Input validation
    if (validate_sim_results and test_input is None and test_truth is None) or \
        (show_quantization_plots and test_input is None):
        raise Exception('Invalid parameter combination. Test data is required.')
        
    # Define all return values as null
    csim_binary_accuracy = None
    csim_auc = None
    cosim_binary_accuracy = None
    cosim_auc = None
    syn_xml = None
    sim_rpt = None
    
    # Try-catch block used to always move the cwd back to the original
    try:
        with open('config_template.yml', 'r') as fin:
            cfg_template = fin.read()

        # Change cwd into synthesis dir
        path = f'{syn_dir}/{model.name}'
        if os.path.exists(path):
            shutil.rmtree(path)
        
        os.makedirs(path)
        os.chdir(path)

        # Write model.h5 file
        model.save(f'{model.name}.h5')

        # Write test data to .dat
        if validate_sim_results:
            test_data_prefix = ''
            os.makedirs(test_data_dir, exist_ok=True)
            with open(test_data_dir + '/test_images.dat', 'w') as input_out, open(test_data_dir + '/test_labels.dat', 'w') as truth_out:
                input_out.write('\n'.join([' '.join(x.flatten()) for x in test_input.astype(str)]))
                truth_out.write('\n'.join(test_truth.astype(str)))
        else:
            test_data_prefix = '#'
            
        # Parse formatted cfg
        cfg = yaml.safe_load(cfg_template.format(name=model.name, io_type=io_type, ap_type=ap_type, 
                                                 strategy=strategy, test_data_prefix=test_data_prefix))
                
        # Convert model
        hls_model = hls4ml.converters.keras_to_hls(cfg)
        hls_model.compile()
        
        if show_quantization_plots:
            ap, wp = hls4ml.model.profiling.numerical(keras_model=model, hls_model=hls_model, X=test_input)
            plt.show()
    
        # Synthesize Model
        hls_model.build(csim=csim, synth=True, cosim=cosim)
        
        if validate_sim_results:
            if csim:
                csim_binary_accuracy, csim_auc = calculate_metrics(test_truth, f'{model.name}/tb_data/csim_results.log')
                print('-'*50)
                print(model.name + ' CSIM Metrics')
                print(f'binary accuracy = {csim_binary_accuracy}')
                print(f'AUC = {csim_auc}')
                print('-'*50)
            if cosim:
                cosim_binary_accuracy, cosim_auc = calculate_metrics(test_truth, f'{model.name}/tb_data/rtl_cosim_results.log')
                print('-'*50)
                print(model.name + ' CO-SIM Metrics')
                print(f'binary accuracy = {cosim_binary_accuracy}')
                print(f'AUC = {cosim_auc}')
                print('-'*50)
            
        # grab xml data for synthesis (and possbily cosim)
        with open(f'{model.name}/{model.name}_prj/solution1/syn/report/csynth.xml', 'rb') as fin:
            syn_xml = xmltodict.parse(fin)
        
        if cosim:
            sim_rpt = parse_sim_rpt(f'{model.name}/{model.name}_prj/solution1/sim/report/verilog/lat.rpt')
                
        
        # Move back into previous wd
        os.chdir('../..')
        
        if delete_files:
            shutil.rmtree(path)
        
    except:
        os.chdir('../..')
        raise
    
    return {
        'output_metrics': {
            'csim': {
                'binary_accuracy': csim_binary_accuracy,
                'auc': csim_auc
            },
            'cosim': {
                'binary_accuracy': cosim_binary_accuracy,
                'auc': cosim_auc
            }
        },
        'reports': {
            'syn_xml': syn_xml,
            'sim_rpt': sim_rpt
        },
        'syn_latency_estimates': {
            'best': syn_xml['profile']['PerformanceEstimates']['SummaryOfOverallLatency']['Best-caseLatency'],
            'avg': syn_xml['profile']['PerformanceEstimates']['SummaryOfOverallLatency']['Average-caseLatency'],
            'worst': syn_xml['profile']['PerformanceEstimates']['SummaryOfOverallLatency']['Worst-caseLatency']
        }
    }